Create model

In [1]:
import gurobipy as gb
import numpy as np
import pandas as pd

In [2]:
direction = gb.GRB.MINIMIZE #Min / Max

# Create a Gurobi model       
m = gb.Model()

# Add variables
x = m.addVars(2, lb=0, ub=gb.GRB.INFINITY, name="x")

# Set objective function
obj = x[0] ** 2 + x[1] ** 2
m.setObjective(obj, direction) #Minimization / Maximization

m.addConstr(x[0] + x[1] == 4)
m.addConstrs(x[i] >= 0 for i in range(2))

m.update()
m.display()

print('-----------------------------------------------')

m.optimize()

if m.status == gb.GRB.OPTIMAL:
    constraints = m.getConstrs()
    #The constraint dual value / sensitivity in the current solution (also known as the shadow price)... https://www.gurobi.com/documentation/9.5/refman/pi.html
    n_constr = len(constraints)
    optimized_sens = [constraints[k].Pi for k in range(n_constr)]
    print('-----------------------------------------------')
    print("Optimal objective value: %.2f" % m.objVal)

    print(x[0].VarName + ": %.2f" % x[0].x)
    print(x[1].VarName + ": %.2f" % x[1].x)
    print("Dual Variables:")
    
    for i in range(n_constr):
        print(i + 1, ": %.2f" % optimized_sens[i])


else:
    print("Optimization was not successful.")     
  

# Dispose of the Gurobi model to release resources
m.dispose()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-28
Minimize
  0.0 + [ x[0] ^ 2 + x[1] ^ 2 ]
Subject To
  R0: x[0] + x[1] = 4
  R1: x[0] >= 0
  R2: x[1] >= 0
-----------------------------------------------
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xdf5d0cea
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 4e+00]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00

### Lagrangian Relaxation

In [33]:
a = 1
b = 0.5
tolerance = 0.001
lambda_ini = -1
max_iterations = 1000
finished = False
n = 0
x_res = [0, 0]

lambda_i = lambda_ini

direction = gb.GRB.MINIMIZE

while (not finished):
    for i in range(2): #to avoid repeating code
        m = gb.Model()
        m.setParam('OutputFlag', 0) #mute printouts
        x = m.addVar(lb=0, ub=gb.GRB.INFINITY, name="x1")
        obj = x ** 2 + lambda_i * x
        m.setObjective(obj, direction)
        m.addConstr(x >= 0)

        m.optimize()

        if m.status == gb.GRB.OPTIMAL:
            x_res[i] = x.x
        else:
            print("Optimization was not successful.")     
        
        # Dispose of the Gurobi model to release resources
        m.dispose()

    #Check convergence - if not, step
    lambda_next = lambda_i + (1 / (a + b * n)) * (x_res[0] + x_res[1] - 4) / abs(x_res[0] + x_res[1] - 4)
    
    try:
        error = abs(lambda_next - lambda_i)/abs(lambda_i) #avoid division by zero
    except:
        error = 1

    print('\n-----------------------------------------------\n')
    print("Iteration %d..." % n)
    print("Lambda: ", lambda_next, " Error: ", error)

    if error <= tolerance:
        finished = True
        print("\nConverged at iteration %d!" % n)
        print("[x1, x2] = ", x_res)

    elif n >= max_iterations:
        finished = True
        print("\nNo convergence!")

    else:
        lambda_i = lambda_next
        n += 1


-----------------------------------------------

Iteration 0...
Lambda:  -2.0  Error:  1.0

-----------------------------------------------

Iteration 1...
Lambda:  -2.6666666666666665  Error:  0.33333333333333326

-----------------------------------------------

Iteration 2...
Lambda:  -3.1666666666666665  Error:  0.1875

-----------------------------------------------

Iteration 3...
Lambda:  -3.5666666666666664  Error:  0.1263157894736842

-----------------------------------------------

Iteration 4...
Lambda:  -3.9  Error:  0.0934579439252337

-----------------------------------------------

Iteration 5...
Lambda:  -4.185714285714286  Error:  0.07326007326007335

-----------------------------------------------

Iteration 6...
Lambda:  -3.935714285714286  Error:  0.05972696245733788

-----------------------------------------------

Iteration 7...
Lambda:  -4.157936507936508  Error:  0.056462996571889516

-----------------------------------------------

Iteration 8...
Lambda:  -3.95

### Augmented Lagrangian Relaxation

In [35]:
gamma = 1
tolerance = 0.000001
lambda_ini = -1
max_iterations = 100
finished = False
n = 0
x_res = [0, 0]

lambda_i = lambda_ini

direction = gb.GRB.MINIMIZE

x_res_prev = x_res.copy() # for the first iteration

while (not finished):
    for i in range(2): #to avoid repeating code
        m = gb.Model()
        m.setParam('OutputFlag', 0) #mute printouts
        x = m.addVar(lb=0, ub=gb.GRB.INFINITY, name="x1")

        #Using if-statement to differentiate between the two variables - for which should be fixed
        if i == 0:
            obj = x ** 2 + lambda_i * x + ((x_res_prev[1] + x - 4) ** 2) * gamma / 2
        else:
            obj = x ** 2 + lambda_i * x + ((x_res_prev[0] + x - 4) ** 2) * gamma / 2

        m.setObjective(obj, direction)
        m.addConstr(x >= 0)

        m.optimize()

        if m.status == gb.GRB.OPTIMAL:
            x_res[i] = x.x
        else:
            print("Optimization was not successful.")     
        
        # Dispose of the Gurobi model to release resources
        m.dispose()
    
    x_res_prev = x_res.copy() #save the current values for x into the "previos" list for the next loop

    #Check convergence - if not, step
    lambda_next = lambda_i + gamma * (x_res[0] + x_res[1] - 4)
    
    try:
        error = abs(lambda_next - lambda_i)/abs(lambda_i) #avoid division by zero
    except:
        error = 1

    print('\n-----------------------------------------------\n')
    print("Iteration %d..." % n)
    print("Lambda: ", lambda_next, " Error: ", error)

    if error <= tolerance:
        finished = True
        print("\nConverged at iteration %d!" % n)
        print("[x1, x2] = ", x_res)

    elif n >= max_iterations:
        finished = True
        print("\nNo convergence!")

    else:
        lambda_i = lambda_next
        n += 1


-----------------------------------------------

Iteration 0...
Lambda:  -1.6666666666666665  Error:  0.6666666666666665

-----------------------------------------------

Iteration 1...
Lambda:  -3.0  Error:  0.8000000000000002

-----------------------------------------------

Iteration 2...
Lambda:  -3.2222222222222214  Error:  0.0740740740740738

-----------------------------------------------

Iteration 3...
Lambda:  -3.6666666666666665  Error:  0.13793103448275884

-----------------------------------------------

Iteration 4...
Lambda:  -3.7407407407407405  Error:  0.020202020202020173

-----------------------------------------------

Iteration 5...
Lambda:  -3.888888888888888  Error:  0.039603960396039424

-----------------------------------------------

Iteration 6...
Lambda:  -3.91358024691358  Error:  0.006349206349206493

-----------------------------------------------

Iteration 7...
Lambda:  -3.962962962962963  Error:  0.012618296529968511

---------------------------------